# LangChain
## ToolCalling
- LLM이 외부 기능이나 데이터에 접근할 수 있게 해주는 매커니즘
![](images/tool_call.png)

In [1]:
import re
import os, json
import warnings
from datetime import datetime
from pprint import pprint
from textwrap import dedent
from dotenv import load_dotenv
from langchain_community.document_loaders import WikipediaLoader
from langchain_core.messages import ToolMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig
from sqlalchemy.orm.collections import collection
from sqlalchemy.testing.suite.test_reflection import metadata

warnings.filterwarnings("ignore")
load_dotenv()

True

## 랭체인 내장 도구
- 랭체인은 검색, 코드 인터프리터, 생산성 도구 제공
### Tavily 도구 직접 사용해보기
- [docs](https://python.langchain.com/docs/integrations/tools/tavily_search/)

In [2]:
from langchain_community.tools import TavilySearchResults

query = "최근 한 달 동안 나온 테슬라와 구글 CEO 관련 영문 기사들을 각각 자연스럽게 한국어로 요약해서 Bullet Point로 정리해줘"

tavily_search = TavilySearchResults(max_results=2)
search_results = tavily_search.invoke(query)
search_results

[{'title': '[시사영어 1일1문] 머스크의 테슬라 사면초가에 (NYT ... - YouTube',
  'url': 'https://www.youtube.com/watch?v=IN1lZzNk2O8',
  'content': '[시사영어 1일1문] 머스크의 테슬라 사면초가에 (NYT) (최신영어뉴스로 영어공부) \n 상상영어 \n 136 likes \n 1621 views \n 23 Apr 2025 \n 출처: https://www.nytimes.com/2025/04/22/business/tesla-earnings-elon-musk.html\n(긴 뉴스기사에서 필수적인 내용만을 추출하여 2~3 페이지로 구성했습니다. 완전한 전체 기사는 출처를 참고하시기 바랍니다.) [...] Mr. Musk has said the company’s future is in artificial intelligence technology that will allow Tesla vehicles to drive themselves without human intervention, enabling fleets of Tesla “Cybercabs” to make money ferrying customers.\nBut Tesla has not yet perfected the technology and faces competition in that nascent business from several Chinese companies and Waymo, a unit of Alphabet, the parent company of Google. [...] [본문]\nTesla’s 71% Drop in Profit May Pressure Elon Musk to Return to Day Job\nTesla said Tuesday that its profits fell 71 percent in the first three months of the year, which could increase t

### Tool의 구성 요소
- name : 도구의 이름
- description : 도구에 대한 설명
- JSON schema : 도구 입력 정보
- function : 실행할 함수??

In [3]:
TavilySearchResults # class 살펴보기
type(tavily_search)

langchain_community.tools.tavily_search.tool.TavilySearchResults

In [4]:
tavily_search.name

'tavily_search_results_json'

In [5]:
tavily_search.description

'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.'

In [6]:
tavily_search.args

{'query': {'description': 'search query to look up',
  'title': 'Query',
  'type': 'string'}}

In [7]:
tavily_search.args_schema

langchain_community.tools.tavily_search.tool.TavilyInput

In [8]:
tavily_search.args_schema.model_json_schema()

{'description': 'Input for the Tavily tool.',
 'properties': {'query': {'description': 'search query to look up',
   'title': 'Query',
   'type': 'string'}},
 'required': ['query'],
 'title': 'TavilyInput',
 'type': 'object'}

### LLM과 Tavily 도구 binding하기

In [9]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools=[tavily_search])

query

'최근 한 달 동안 나온 테슬라와 구글 CEO 관련 영문 기사들을 각각 자연스럽게 한국어로 요약해서 Bullet Point로 정리해줘'

In [10]:
ai_msg = llm_with_tools.invoke(query)
pprint(ai_msg)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8xxSLkQGxnP4v2ENd4Jigpo1', 'function': {'arguments': '{"query": "Tesla CEO news last month"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_8tGMg1A3iTVofiSJQ1cqiX4F', 'function': {'arguments': '{"query": "Google CEO news last month"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 113, 'total_tokens': 174, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BVccOtmhxRbWKwLonxfMgC59dJYaC', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-9da5005f-17e8-4e63-8511-ec3200cca653-0', tool_calls=[{'name': 'tavily_search_results_json', 'a

In [11]:
ai_msg.content

''

In [12]:
ai_msg.tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'Tesla CEO news last month'},
  'id': 'call_8xxSLkQGxnP4v2ENd4Jigpo1',
  'type': 'tool_call'},
 {'name': 'tavily_search_results_json',
  'args': {'query': 'Google CEO news last month'},
  'id': 'call_8tGMg1A3iTVofiSJQ1cqiX4F',
  'type': 'tool_call'}]

### LLM에서 응답으로 온 Tool 정보를 이용해 Tool 직접 실행하기
- args 스키마 사용 -> list 형태로 나옴
- tool_call 사용 -> ToolMessage 형태로 나옴

In [13]:
# args 스키마 사용
tool_call = ai_msg.tool_calls[0]
tool_output = tavily_search.invoke(tool_call['args'])
pprint(type(tool_output))
pprint(tool_output)

<class 'list'>
[{'content': 'The outlet also reported that the Tesla board of directors last '
             'month began a formal search to replace Musk as its CEO following '
             'continued struggles for the company, as well as Musk’s continued '
             'involvement in President Donald Trump’s administration. Tesla '
             'denied the move, including to WSJ prior to the report’s '
             'publication. [...] Tesla’s struggles have only worsened since '
             'Musk reportedly considered stepping down last year. Tesla’s '
             'share price has tumbled 41% since its December peak, and the EV '
             'maker reported a 20% plunge in first-quarter year-over-year '
             'automotive revenue last month. [...] Oops, something went wrong\n'
             '\n'
             'News\n'
             '\n'
             'Life\n'
             '\n'
             'Entertainment\n'
             '\n'
             'Finance\n'
             '\n'
            

In [14]:
# tool_call 사용
tool_message = tavily_search.invoke(tool_call)
pprint(type(tool_message))
pprint(tool_message)

<class 'langchain_core.messages.tool.ToolMessage'>
ToolMessage(content='[{"title": "Elon Musk reportedly said last year he no longer wanted to be Tesla ...", "url": "https://finance.yahoo.com/news/elon-musk-reportedly-said-last-111700409.html", "content": "The outlet also reported that the Tesla board of directors last month began a formal search to replace Musk as its CEO following continued struggles for the company, as well as Musk’s continued involvement in President Donald Trump’s administration. Tesla denied the move, including to WSJ prior to the report’s publication. [...] Tesla’s struggles have only worsened since Musk reportedly considered stepping down last year. Tesla’s share price has tumbled 41% since its December peak, and the EV maker reported a 20% plunge in first-quarter year-over-year automotive revenue last month. [...] Oops, something went wrong\\n\\nNews\\n\\nLife\\n\\nEntertainment\\n\\nFinance\\n\\nSports\\n\\nNew on Yahoo\\n\\nYahoo Finance\\n\\nIn This Article

In [15]:
# tool_output으로 ToolMessage 만들기
from langchain_core.messages.tool import ToolMessage

tool_message = ToolMessage(
    content=tool_output,
    tool_call_id=tool_call["id"],
    name=tool_call["name"]
)
pprint(type(tool_message))
pprint(tool_message)

<class 'langchain_core.messages.tool.ToolMessage'>
ToolMessage(content=[{'title': 'Elon Musk reportedly said last year he no longer wanted to be Tesla ...', 'url': 'https://finance.yahoo.com/news/elon-musk-reportedly-said-last-111700409.html', 'content': 'The outlet also reported that the Tesla board of directors last month began a formal search to replace Musk as its CEO following continued struggles for the company, as well as Musk’s continued involvement in President Donald Trump’s administration. Tesla denied the move, including to WSJ prior to the report’s publication. [...] Tesla’s struggles have only worsened since Musk reportedly considered stepping down last year. Tesla’s share price has tumbled 41% since its December peak, and the EV maker reported a 20% plunge in first-quarter year-over-year automotive revenue last month. [...] Oops, something went wrong\n\nNews\n\nLife\n\nEntertainment\n\nFinance\n\nSports\n\nNew on Yahoo\n\nYahoo Finance\n\nIn This Article:\n\nTesla has op

In [16]:
tool_message.content

[{'title': 'Elon Musk reportedly said last year he no longer wanted to be Tesla ...',
  'url': 'https://finance.yahoo.com/news/elon-musk-reportedly-said-last-111700409.html',
  'content': 'The outlet also reported that the Tesla board of directors last month began a formal search to replace Musk as its CEO following continued struggles for the company, as well as Musk’s continued involvement in President Donald Trump’s administration. Tesla denied the move, including to WSJ prior to the report’s publication. [...] Tesla’s struggles have only worsened since Musk reportedly considered stepping down last year. Tesla’s share price has tumbled 41% since its December peak, and the EV maker reported a 20% plunge in first-quarter year-over-year automotive revenue last month. [...] Oops, something went wrong\n\nNews\n\nLife\n\nEntertainment\n\nFinance\n\nSports\n\nNew on Yahoo\n\nYahoo Finance\n\nIn This Article:\n\nTesla has opened a formal search to replace CEO Elon Musk, the Wall Street Jour

### Tool의 결과를 LLM에 전달하여 최종 응답 생성하기

In [17]:
from datetime import datetime

today = datetime.today().strftime("%Y-%m-%d")
llm = ChatOpenAI(model="gpt-4o-mini")
prompt = f"""
    You are a helpful AI assistant.
    Today is {today}
    {query}
    {tool_message.content}
"""
result = llm.invoke(prompt)
pprint(result)

AIMessage(content='### 테슬라 관련 기사 요약\n\n- **엘론 머스크, CEO 역할 축소 중**: 머스크는 지난해 더 이상 테슬라 CEO 역할을 원하지 않는다고 언급했으며, 최근 테슬라 이사회는 CEO 교체를 위한 공식 검색을 시작한 것으로 보도됨. \n- **테슬라의 재정적 어려움**: 테슬라는 채산성이 떨어지고 있으며, 지난해 12월 정점 대비 주가가 41% 하락했고, 1분기 자동차 매출이 전년 대비 20% 감소했음.\n- **이사회 부인**: 테슬라 이사회는 새로운 CEO를 찾고 있다는 보도를 부인하며, 머스크가 지금은 다른 일에 집중하고 있다고 전함.\n\n### 구글 관련 기사 요약\n\n현재 구글 관련 기사 요약에 대한 정보는 제공되지 않아, 구글 CEO 관련 정보를 추가로 제공해 주시면 요약해드리겠습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 207, 'prompt_tokens': 443, 'total_tokens': 650, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'id': 'chatcmpl-BVccVGTfvYPVnB5REd1ENUdMQVAhe', 'finish_reason': 'stop', 'logprobs': None}, id='run-24aa9c18-f6d2-41de-88e6-5cdba0fc4a47-0', usage_metadat

## 전체과정 정리 하기
- LLM 체인 정의
- LLM 체인에 ToolMessage 전달

In [18]:
# LLM 체인 정의

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant"),
    ("system", f"Today is {today}"),
    ("human", "{user_input}"),
    ("placeholder", "{messages}")
])

# llm 정의
llm = ChatOpenAI(model="gpt-4o-mini")

# 도구 바인딩
llm_with_tools = llm.bind_tools(tools=[tavily_search])

# LLM체인 생성
llm_chain = prompt | llm_with_tools


In [19]:
# LLM 체인에 ToolMessage 전달
from langchain_core.runnables import chain

@chain
def web_search_chain(user_input: str, config: RunnableConfig):
    input = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input, config=config)
    tool_msgs = tavily_search.batch(ai_msg.tool_calls, config=config)
    return llm_chain.invoke(
        {**input, "messages": [ai_msg, *tool_msgs]},
        config=config
    )

web_search_chain.invoke(query)

AIMessage(content='### 테슬라 CEO 관련 최근 뉴스 요약\n- **일론 머스크 CEO 교체 보도 부인**: 테슬라 이사회 의장 로빈 덴홀름이 최근 보도와 관련하여, 테슬라 이사회가 CEO 교체를 위한 모집업체에 연락했다는 주장은 사실이 아니며, 머스크가 CEO로서 계속해서 회사를 이끌 것이라고 발표함.\n- **이사회에서의 탐색적 단계**: 월스트리트저널에 따르면, 테슬라 이사회는 몇 주 전부터 적절한 후임자를 찾기 위한 탐색적인 단계를 진행하고 있었고, 머스크에게 회사에 다시 집중할 필요가 있다고 경고한 것으로 전해짐.  \n\n### 구글 CEO 관련 최근 뉴스 요약\n- **AI 및 양자 혁신 강조**: 구글 CEO 순다르 피차이는 AI와 양자 기술 혁신이 미래의 핵심이 될 것이라고 강조함.\n- **AI 팀 재조정 및 딥마인드 통합**: 구글 CEO는 2025년을 대비해 AI 제품 및 서비스를 새롭게 출시할 계획을 직원들에게 알림. Gemini AI 모델에 큰 기대를 걸고 있으며, 이 모델이 5억 사용자에 도달할 수 있도록 목표하고 있음.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 281, 'prompt_tokens': 1339, 'total_tokens': 1620, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BVccd0

## 사용자 정의 도구(Tool)
- @tool 데코레이터 사용

In [20]:
from typing import Dict, List, Annotated
import requests
from langchain_core.tools import tool

@tool
def blog_search(query:  Annotated[str, "질의"], search_count: Annotated[int, "검색수"]=2) -> List[Dict]:
    """네이버 블로그 API에 검색 요청을 보냅니다."""
    url = "https://openapi.naver.com/v1/search/blog.json"
    headers = {
        "X-Naver-Client-Id": os.environ["NAVER_CLIENT_ID"],
        "X-Naver-Client-Secret": os.environ["NAVER_CLIENT_SECRET"]
    }
    params = {"query": query, "display": 10, "start": 1}
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()['items']
    else:
        return []

wine_query = "블로그에 나와있는 스테이크와 어울리는 와인과 기분 좋은 노래를 추천해주세요."
search_results = blog_search.invoke(wine_query)
search_results

[{'title': '목동<b>와인</b> - 파인앨리 : 뽈뽀 <b>스테이크</b>, 감베리 풍기 알리오',
  'link': 'https://blog.naver.com/offspring84/223860446265',
  'description': '맛<b>있는</b> 파스타, <b>와인</b>도 한 잔 <b>추천</b>드립니다 술 안 마시는 친구랑 와도 좋고 연인과 즐거운 데이트 코스로도 <b>좋은</b> 곳! 목동<b>와인</b> - 파인앨리: 뽈뽀 <b>스테이크</b>, 감베리 풍기 알리오, 스몰카프레제, Veuve de Verndier <b>와인</b>',
  'bloggername': '무한잡념',
  'bloggerlink': 'blog.naver.com/offspring84',
  'postdate': '20250509'},
 {'title': '이탈리아 피렌체 맛집 <b>추천</b> 티본<b>스테이크</b> 토스카나 와인 육회',
  'link': 'https://blog.naver.com/lsh5755/223847077153',
  'description': "피렌체 인근 토스카나 지방은 질 <b>좋은</b> <b>와인</b>을 생산하기로 유명하다. <b>와인</b> 애호가들의 성지기도 한다.... 그냥 'T본 <b>스테이크</b>'라고 부르고, 그들도 알아듣는다. T자 뼈가 포함되어 <b>있는</b> 안심과 등심 부위를... ",
  'bloggername': '천사는 여행중',
  'bloggerlink': 'blog.naver.com/lsh5755',
  'postdate': '20250426'},
 {'title': '한우 <b>스테이크</b>를 <b>와인과</b> 함께 즐길 수 <b>있는</b> 이도현 고깃집',
  'link': 'https://blog.naver.com/commitv/223848860811',
  'description': '총평 광교 <b>스테이크</b> 맛집 이도현 고깃집에서 <b>기분 좋은</b> 한 끼 식사

In [21]:
pprint(blog_search.name)
pprint(blog_search.description)
pprint(blog_search.args_schema)
pprint(blog_search.args)

'blog_search'
'네이버 블로그 API에 검색 요청을 보냅니다.'
<class 'langchain_core.utils.pydantic.blog_search'>
{'query': {'description': '질의', 'title': 'Query', 'type': 'string'},
 'search_count': {'default': 2,
                  'description': '검색수',
                  'title': 'Search Count',
                  'type': 'integer'}}


In [22]:
blog_search

StructuredTool(name='blog_search', description='네이버 블로그 API에 검색 요청을 보냅니다.', args_schema=<class 'langchain_core.utils.pydantic.blog_search'>, func=<function blog_search at 0x000001F84CC7AAC0>)

In [23]:
# LLM binding
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_blog = llm.bind_tools(tools=[tavily_search, blog_search])

In [24]:
# LLM 실행하여 툴 추천 받기
ai_msg = llm_with_blog.invoke(wine_query)
ai_msg

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7oJQmz6ubJgcSZPeqFQZao8O', 'function': {'arguments': '{"query": "스테이크와 어울리는 와인"}', 'name': 'blog_search'}, 'type': 'function'}, {'id': 'call_ZPdn9KcKkjDStSHzAvYQUE6R', 'function': {'arguments': '{"query": "기분 좋은 노래"}', 'name': 'blog_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 149, 'total_tokens': 206, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BVccjUMJkpBxztqbp3bQ4yicXLV7s', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4b0b65f2-4e28-4b51-9a24-156e7584290f-0', tool_calls=[{'name': 'blog_search', 'args': {'query': '스테이크와 어울리는 와인'}, 'id': 'call_7oJQmz6ubJgcSZPeqFQZao8O', 't

In [25]:
ai_msg.tool_calls

[{'name': 'blog_search',
  'args': {'query': '스테이크와 어울리는 와인'},
  'id': 'call_7oJQmz6ubJgcSZPeqFQZao8O',
  'type': 'tool_call'},
 {'name': 'blog_search',
  'args': {'query': '기분 좋은 노래'},
  'id': 'call_ZPdn9KcKkjDStSHzAvYQUE6R',
  'type': 'tool_call'}]

In [26]:
# blog tool 실행
blog_search.batch(ai_msg.tool_calls)

[ToolMessage(content='[{"title": "스미스앤월렌스키 (<b>스테이크와 어울리는</b> 추천<b>와인</b>, 서울... ", "link": "https://blog.naver.com/neotaboo/223758018557", "description": "해주셔서 <b>와인</b>과 함께 <b>스테이크</b> 먹기 너무 좋은 매장이었다. 식전빵이 나왔다. 로즈메리를 가득 뿌려... 잘 <b>어울리는</b> 수프였다. 꾸덕한 식감도 좋고, 풍부한 크림맛도 좋고. 입 짧은 나도 처음으로 접시 전체를... ", "bloggername": "입 짧은 콩나물", "bloggerlink": "blog.naver.com/neotaboo", "postdate": "20250212"}, {"title": "<b>스테이크와 어울리는 와인</b> 추천과 종류", "link": "https://blog.naver.com/oipicle31/223708302326", "description": "Information <b>스테이크</b> <b>어울리는 와인</b> 추천 <b>스테이크와</b> <b>와인</b>, 이 둘의 조합은 마치 해변의 파도와 모래처럼 완벽하게 <b>어울리는</b> 궁합이라고 할 수 있죠. 오늘은 여러분의 저녁 식사를 더욱 특별하게 만들어줄... ", "bloggername": "별난하루", "bloggerlink": "blog.naver.com/oipicle31", "postdate": "20241229"}, {"title": "[마곡 <b>와인</b> / 마곡 파스타] 마곡 <b>스테이크와</b> <b>와인</b>이 <b>어울리는</b>... ", "link": "https://blog.naver.com/candytop100/223163377277", "description": "<b>와인</b>은 추천을 받아 레드 <b>와인</b> 중 너무 달지는 않은데, <b>스테이크와</b> 잘 <b>어울리는 와인</b>으로 추천을 받았어용~!! A

## Runnable 객체를 도구(tool) 변환
* 무자열이나 dict 입력을 받는 Runnable을 도구로 변환
* as_tool 메서드 사용

In [27]:
from langchain_core.runnables import RunnableLambda
from pydantic import BaseModel, Field
from langchain_core.documents import Document
from langchain_community.document_loaders.wikipedia import WikipediaLoader

class WikiSearchSchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia.")
    k: int = Field(2, description="The number of documents to return (default is 2")

def search_wiki(input_data: dict) -> List[Document]:
    """
    Search Wikipedia documents based on user input (query) and return k documents
    """
    query = input_data["query"]
    k = input_data.get("k", 2)
    wiki_loader = WikipediaLoader(query=query, load_max_docs=k, lang="ko")
    wiki_docs = wiki_loader.load()
    return wiki_docs


runnable = RunnableLambda(search_wiki)
wiki_search = runnable.as_tool(
    name="wiki_search",
    description=dedent("""
        Use this tool when you want to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a specified number of documents. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSearchSchema,
)

In [28]:
pprint(wiki_search)
pprint(wiki_search.name)
pprint(wiki_search.description)
pprint(wiki_search.args)

StructuredTool(name='wiki_search', description="Use this tool when you want to search for information on Wikipedia.\nIt searches for Wikipedia articles related to the user's query and returns\na specified number of documents. This tool is useful when general knowledge\nor background information is required.", args_schema=<class '__main__.WikiSearchSchema'>, func=<function convert_runnable_to_tool.<locals>.invoke_wrapper at 0x000001F84C5AB2E0>, coroutine=<function convert_runnable_to_tool.<locals>.ainvoke_wrapper at 0x000001F84C5AAF20>)
'wiki_search'
('Use this tool when you want to search for information on Wikipedia.\n'
 "It searches for Wikipedia articles related to the user's query and returns\n"
 'a specified number of documents. This tool is useful when general knowledge\n'
 'or background information is required.')
{'k': {'default': 2,
       'description': 'The number of documents to return (default is 2',
       'title': 'K',
       'type': 'integer'},
 'query': {'description':

In [29]:
# 도구 호출
query = "파스타의 유래"
wiki_results = wiki_search.invoke({"query": query})

for result in wiki_results:
    pprint(result)

Document(metadata={'title': '오르조', 'summary': '오르조(이탈리아어: orzo) 또는 리소니(이탈리아어: risoni, 단수: risone 리소네[*])는 이탈리아의 파스타이다. "오르조"는 라틴어:hordeum에서 유래했으며 "보리"를 뜻한다. "리소니"는 "큰 쌀"이라는 뜻이다. 파스타의 일종으로 큰 쌀알의 모양을 하고 있으며 솔방울이나 잣보다는 좀 더 작다. 보통 라구 등 수프와 함께 먹는다. 원래는 보리로 만들었지만 요즘에는 박력분으로 만드는 것이 흔해졌다. 다른 이름으로는 kritharáki ("little barley") 혹은 manéstra(그리스 요리)로 부르며, lisān al-`uṣfūr ("명금의 혀")라고 아랍 요리에서 불린다. 오르조를 두고 이탈리아의 쌀이라고 부르기도 한다.\n터키에서는 아르파 셰흐리예(arpa şehriye)로 불리며, 셰흐리예의 하나이다. 필라브를 만들거나 초르바(수프)에 넣어 먹는다.', 'source': 'https://ko.wikipedia.org/wiki/%EC%98%A4%EB%A5%B4%EC%A1%B0'}, page_content='오르조(이탈리아어: orzo) 또는 리소니(이탈리아어: risoni, 단수: risone 리소네[*])는 이탈리아의 파스타이다. "오르조"는 라틴어:hordeum에서 유래했으며 "보리"를 뜻한다. "리소니"는 "큰 쌀"이라는 뜻이다. 파스타의 일종으로 큰 쌀알의 모양을 하고 있으며 솔방울이나 잣보다는 좀 더 작다. 보통 라구 등 수프와 함께 먹는다. 원래는 보리로 만들었지만 요즘에는 박력분으로 만드는 것이 흔해졌다. 다른 이름으로는 kritharáki ("little barley") 혹은 manéstra(그리스 요리)로 부르며, lisān al-`uṣfūr ("명금의 혀")라고 아랍 요리에서 불린다. 오르조를 두고 이탈리아의 쌀이라고 부르기도 한다.\n터키에서는 아르파 셰흐리예(arpa şehriye)로 불리며, 셰흐리예의 하나이다. 필라브를 만

In [30]:
# LLM에 도구 바인딩
llm_with_tools = llm.bind_tools(tools=[wiki_search, tavily_search, blog_search])

query = "서울 강남의 유명한 파스타 맛집은 어디인가요? 그리고 파스타의 유래를 위키피디아에서 찾아주세요."
ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg)
pprint("-" * 100)

pprint(ai_msg.content)
pprint("-" * 100)

pprint(ai_msg.tool_calls)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_w8XFxyknTYQb1st45OssKsK0', 'function': {'arguments': '{"query": "서울 강남 파스타 맛집", "search_count": 5}', 'name': 'blog_search'}, 'type': 'function'}, {'id': 'call_xaEQAZc0qzDytm5Li3nCdTta', 'function': {'arguments': '{"query": "파스타", "k": 2}', 'name': 'wiki_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 248, 'total_tokens': 310, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_96c46af214', 'id': 'chatcmpl-BVccst5469AWJxVF8nZXm0laI8FQZ', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6290d6e0-67dc-4f3e-8f6e-f7d0484e49d1-0', tool_calls=[{'name': 'blog_search', 'args': {'query': '서울 강남 파스타 맛집', 'search_count': 5}, 'i

## LCEL 체인을 도구로 변환하기

In [31]:
def wiki_search_and_summarize(input_data: dict):
    wiki_loader = WikipediaLoader(query=input_data["query"], load_max_docs=2, lang="ko")
    wiki_docs = wiki_loader.load()

    formatted_docs = [
        f'<Document source="{doc.metadata["source"]}">\n{doc.page_content}\n</Document>'
        for doc in wiki_docs
    ]
    return formatted_docs

summary_prompt = ChatPromptTemplate.from_template(
    "Summarize the following text in a concise manner:\n\n{context}\n\nSummary:"
)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
summary_chain = (
    {"context": RunnableLambda(wiki_search_and_summarize)}
    | summary_prompt | llm | StrOutputParser()
)

summarized_text = summary_chain.invoke({"query": "파스타의 유래"})
pprint(summarized_text)

('오르조(또는 리소니)는 이탈리아의 파스타로, 보리에서 유래된 이름을 가지고 있으며 큰 쌀알 모양이다. 주로 수프와 함께 먹으며, 현재는 '
 '박력분으로 만들어진다. 터키에서는 아르파 셰흐리예로 불리며, 필라브나 수프에 사용된다. 이탈리아 요리는 기원전 4세기부터 발전해왔으며, '
 '지역마다 특색이 다르다. 북부는 쌀과 유제품을, 남부는 올리브와 해산물을 주로 사용한다. 이탈리아 요리는 간결함과 재료의 질을 중시하며, '
 '2013년 CNN에서 세계 최고의 요리로 선정되었다.')


In [32]:
# wiki summary chain을 도구로 변환하기
class WikiSummarySchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")

wiki_summary_search = summary_chain.as_tool(
    name="wiki_summary",
    description=dedent("""
        Use this tool when you want to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a summarized text. this tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSummarySchema,
)

pprint(type(wiki_summary_search))
pprint("-"*100)
pprint(wiki_summary_search.name)
pprint("-"*100)
pprint(wiki_summary_search.description)
pprint("-"*100)
pprint(wiki_summary_search.args)
pprint("-"*100)
pprint(wiki_summary_search.args_schema)
pprint("-"*100)
pprint(wiki_summary_search.args_schema.schema())

<class 'langchain_core.tools.structured.StructuredTool'>
'----------------------------------------------------------------------------------------------------'
'wiki_summary'
'----------------------------------------------------------------------------------------------------'
('Use this tool when you want to search for information on Wikipedia.\n'
 "It searches for Wikipedia articles related to the user's query and returns\n"
 'a summarized text. this tool is useful when general knowledge\n'
 'or background information is required.')
'----------------------------------------------------------------------------------------------------'
{'query': {'description': 'The query to search for in Wikipedia',
           'title': 'Query',
           'type': 'string'}}
'----------------------------------------------------------------------------------------------------'
<class '__main__.WikiSummarySchema'>
'------------------------------------------------------------------------------------------

In [33]:
# tavily + blog + wiki_summary
llm_with_tools = llm.bind_tools(tools=[wiki_summary_search, tavily_search, blog_search])
query = "서울 강남의 유명한 파스타 맛집은 어디인가요? 그리고 파스타의 유래를 알려주세요."
ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg)
print("-"*100)
pprint(ai_msg.content)
print("-"*100)
pprint(ai_msg.tool_calls)
print("-"*100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3xVY86Ohn9LfF234DMsBl8oU', 'function': {'arguments': '{"query": "서울 강남 파스타 맛집", "search_count": 5}', 'name': 'blog_search'}, 'type': 'function'}, {'id': 'call_LKR9iFW3vkXenk4qFsqCJleV', 'function': {'arguments': '{"query": "파스타"}', 'name': 'wiki_summary'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 218, 'total_tokens': 276, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'id': 'chatcmpl-BVcd4SRdR9QJQJozuIOXHPWSbi9ry', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-df0dc7cc-a622-4433-ac7f-1672a9b62f09-0', tool_calls=[{'name': 'blog_search', 'args': {'query': '서울 강남 파스타 맛집', 'search_count': 5}, 'id': 'ca

In [34]:
# 도구 호출
tool_message = wiki_summary_search.invoke(ai_msg.tool_calls[1])
pprint(tool_message)

ToolMessage(content='파스타는 이탈리아의 주요 밀 식품으로, 듀럼밀 세몰라와 물 또는 밀가루와 달걀로 만들어지며, 삶거나 구워서 먹는다. 이탈리아의 국민 음식으로 여겨지며, 역사적으로 그리스와 아랍 문화의 영향을 받았다. 파스타는 건파스타와 생파스타로 나뉘며, 다양한 형태와 종류가 존재한다. 요리 방법으로는 삶거나 오븐에 구워내는 방식이 있다. 투움바 파스타는 미국의 아웃백 스테이크하우스에서 유래된 요리로, 매콤한 크림 소스를 사용하며 한국에서 인기를 끌고 있다.', name='wiki_summary', tool_call_id='call_LKR9iFW3vkXenk4qFsqCJleV')


In [35]:
from langchain_core.runnables.base import chain

# 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")
# 프로프트 템플릿
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}"),
    ("human", "{user_input}"),
    ("placeholder", "{messages}")
])
# LLM 도구 바인딩
llm_with_tools = llm.bind_tools(tools=[wiki_summary_search])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

@chain
def wiki_summary_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    print("ai msg: \n",ai_msg)
    print("-"*100)
    tool_msgs = wiki_summary_search.batch(ai_msg.tool_calls, config=config)
    print("tool msgs: \n",tool_msgs)
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
response = wiki_summary_chain.invoke("파스타의 유래에 대해서 알려주세요.")

# 응답 출력
pprint(response.content)


ai msg: 
 content='' additional_kwargs={'tool_calls': [{'id': 'call_T4xs0zvnu7S1SQUUTWsir42W', 'function': {'arguments': '{"query":"파스타의 유래"}', 'name': 'wiki_summary'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 120, 'total_tokens': 140, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BVcdF3c4yDE48yFGTHKG4qI3BNAZK', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-bc6a4448-9186-4382-b01d-6ae16036bccb-0' tool_calls=[{'name': 'wiki_summary', 'args': {'query': '파스타의 유래'}, 'id': 'call_T4xs0zvnu7S1SQUUTWsir42W', 'type': 'tool_call'}] usage_metadata={'input_tokens': 120, 'output_tokens': 20, 'total_tokens': 140, 'input_token_details': {'audio': 0, 'cache_read': 0}

## 백터 저장소 검색기

In [36]:
from langchain.document_loaders import TextLoader

# 메뉴판 텍스트 데이터 로드
loader = TextLoader("./data/restaurant_menu.txt", encoding="utf-8")
documents = loader.load()

print(len(documents))
pprint(documents)

1
[Document(metadata={'source': './data/restaurant_menu.txt'}, page_content='1. 시그니처 스테이크\n   • 가격: ₩35,000\n   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스\n   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.\n\n2. 트러플 리조또\n   • 가격: ₩22,000\n   • 주요 식재료: 이탈리아산 아르보리오 쌀, 블랙 트러플, 파르미지아노 레지아노 치즈\n   • 설명: 크리미한 텍스처의 리조또에 고급 블랙 트러플을 듬뿍 얹어 풍부한 향과 맛을 즐길 수 있는 메뉴입니다. 24개월 숙성된 파르미지아노 레지아노 치즈를 사용하여 깊은 맛을 더했으며, 주문 즉시 조리하여 최상의 상태로 제공됩니다.\n\n3. 연어 타르타르\n   • 가격: ₩18,000\n   • 주요 식재료: 노르웨이산 생연어, 아보카도, 케이퍼, 적양파\n   • 설명: 신선한 노르웨이산 생연어를 곱게 다져 아보카도, 케이퍼, 적양파와 함께 섞어 만든 타르타르입니다. 레몬 드레싱으로 상큼한 맛을 더했으며, 바삭한 브리오쉬 토스트와 함께 제공됩니다. 전채요리로 완벽한 메뉴입니다.\n\n4. 버섯 크림 수프\n   • 가격: ₩10,000\n   • 주요 식재료: 양송이버섯, 표고버섯, 생크림, 트러플 오일\n   • 설명: 양송이버섯과 표고버섯을 오랜 시간 정성스레 끓여 만든 크림 수프입니다. 부드러운 텍스처와 깊은 버섯 향이 특징이며, 최상급 트러플 오일을 살짝 뿌려 고급스러운 향을 더했습니다. 파슬리를 곱게 다져 고명으로 올려 제공됩니다.\n\n5. 가든 샐러드\n   • 가격: ₩12,000\n   • 주요 식재료: 유기농 믹스 그린, 체리 토마토, 오이, 당근,

In [37]:
from langchain_core.documents import Document

# 문서 분할
def split_menu_items(document):
    """메뉴 항목을 분리하는 함수"""
    # 정규표현식 정의, "숫자. "로 시작하고, 두 줄 바꿈으로 끝나거나 문서 끝까지인 항목들을 추출
    pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
    menu_items = re.findall(pattern, document.page_content, re.DOTALL)

    menu_documents = []
    for i, item in enumerate(menu_items):
        menu_name = item.split('\n')[0].split('.', 1)[1].strip()
        menu_doc = Document(
            page_content=item.strip(),
            metadata={
                "source": document.metadata["source"],
                "menu_number": i,
                "menu_name": menu_name,
            }
        )
        menu_documents.append(menu_doc)
    return menu_documents

menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)

print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리")
for doc in menu_documents[:2]:
    print(f"menu number: {doc.metadata['menu_number']}")
    print(f"menu name: {doc.metadata['menu_name']}")
    print(f"menu description: {doc.page_content[:100]}...")

총 10개의 메뉴 항목이 처리
menu number: 0
menu name: 시그니처 스테이크
menu description: 1. 시그니처 스테이크
   • 가격: ₩35,000
   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스
   • 설명: 셰프의 특제 시그니처 메뉴로, ...
menu number: 1
menu name: 트러플 리조또
menu description: 2. 트러플 리조또
   • 가격: ₩22,000
   • 주요 식재료: 이탈리아산 아르보리오 쌀, 블랙 트러플, 파르미지아노 레지아노 치즈
   • 설명: 크리미한 텍스처의 리조...


In [38]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma


embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

menu_db = Chroma.from_documents(
    documents=menu_documents,
    embedding=embeddings_model,
    collection_name="restaurant_menu",
    persist_directory="./chroma.db"
)

# Retriever 생성
menu_retriever = menu_db.as_retriever(search_kargs={'k': 2})

# 쿼리 테스트
query = "시그니처 스테이크의 가격과 특징은 무엇인가요?"
docs = menu_retriever.invoke(query)
print(f"검색 결과: {len(docs)}")
for doc in docs:
    print(f"menu number: {doc.metadata['menu_number']}")
    print(f"menu name: {doc.metadata['menu_name']}")


검색 결과: 4
menu number: 0
menu name: 시그니처 스테이크
menu number: 0
menu name: 시그니처 스테이크
menu number: 0
menu name: 시그니처 스테이크
menu number: 0
menu name: 시그니처 스테이크


In [39]:
# wien 벡터 저장소
# 와인 메뉴 텍스트 데이터를 로드
loader = TextLoader("./data/restaurant_wine.txt", encoding="utf-8")
documents = loader.load()

# 메뉴 항목 분리 실행
menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")


# Chroma 인덱스 생성
wine_db = Chroma.from_documents(
    documents=menu_documents,
    embedding=embeddings_model,
    collection_name="restaurant_wine",
    persist_directory="./chroma_db",
)

wine_retriever = wine_db.as_retriever(
    search_kwargs={'k': 2},
)

query = "스테이크와 어울리는 와인을 추천해주세요."
docs = wine_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print()

총 10개의 메뉴 항목이 처리되었습니다.

메뉴 번호: 0
메뉴 이름: 샤토 마고 2015
내용:
1. 샤토 마고 2015
   • 가격: ₩450,000
   • 주요 품종: 카베르네 소비뇽, 메를로, 카베르네 프랑, 쁘띠 베르도
   • 설명: 보르도 메독 지역의 프리미엄 ...

메뉴 번호: 1
메뉴 이름: 돔 페리뇽 2012
내용:
2. 돔 페리뇽 2012
   • 가격: ₩380,000
   • 주요 품종: 샤르도네, 피노 누아
   • 설명: 프랑스 샴페인의 대명사로 알려진 프레스티지 큐베입니다. 시트러스...
검색 결과: 2개
메뉴 번호: 9
메뉴 이름: 그랜지 2016

메뉴 번호: 9
메뉴 이름: 그랜지 2016



In [40]:
# 메뉴 벡터 저장소 로드
menu_db = Chroma(
    embedding_function=embeddings_model,
    collection_name="restaurant_menu",
    persist_directory="./chroma.db",
)

In [41]:
# search_menu tool 정의
class MenuSearchInput(BaseModel):
    query: str = Field(description="The search query for menu information")

@tool
def menu_search(query: str) -> List[Document]:
    """
    Securely retrieve and access authorized restaurant menu information from the encrypted database.
    Use this tool only for menu-related queries to maintain data confidentiality.
    """
    docs = menu_db.similarity_search(query, k=2)
    if len(docs) > 0:
        return docs

    return [Document(page_content="관련 메뉴 정보를 찾을 수 없습니다.")]

In [42]:

# 도구 속성
print("자료형: ")
print(type(menu_search))
print("-"*100)

print("name: ")
print(menu_search.name)
print("-"*100)

print("description: ")
pprint(menu_search.description)
print("-"*100)

print("schema: ")
pprint(menu_search.args)
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
menu_search
----------------------------------------------------------------------------------------------------
description: 
('Securely retrieve and access authorized restaurant menu information from the '
 'encrypted database.\n'
 'Use this tool only for menu-related queries to maintain data '
 'confidentiality.')
----------------------------------------------------------------------------------------------------
schema: 
{'query': {'title': 'Query', 'type': 'string'}}
----------------------------------------------------------------------------------------------------


In [43]:
# wine 벡터 저장소 로드
wine_db = Chroma(
   embedding_function=embeddings_model,
   collection_name="restaurant_wine",
   persist_directory="./chroma_db",
)

# search_wine 정의
@tool
def wine_search(query: str) -> List[Document]:
   """
   Securely retrieve and access authorized restaurant wine information from the encrypted database.
   Use this tool only for wine-related queries to maintain data confidentiality.
   """
   docs = wine_db.similarity_search(query, k=2)
   if len(docs) > 0:
      return docs

   return [Document(page_content="관련 와인 정보를 찾을 수 없습니다.")]

# 도구 속성
print("자료형: ")
print(type(wine_search))
print("-"*100)

print("name: ")
print(wine_search.name)
print("-"*100)

print("description: ")
pprint(wine_search.description)
print("-"*100)

print("schema: ")
pprint(wine_search.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
wine_search
----------------------------------------------------------------------------------------------------
description: 
('Securely retrieve and access authorized restaurant wine information from the '
 'encrypted database.\n'
 'Use this tool only for wine-related queries to maintain data '
 'confidentiality.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Securely retrieve and access authorized restaurant wine '
                'information from the encrypted database.\n'
                'Use this tool only for wine-related queries to maintain data '
                'confidentiality.',
 'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'wine_search',
 'type': 'object'}
----------------

In [44]:
# LLM에 도구를 바인딩 (2개의 도구 바인딩)
llm_with_tools = llm.bind_tools(tools=[menu_search, wine_search])

# 도구 호출이 필요한 LLM 호출을 수행
query = "시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요."
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_QOxtPmNqByTnibexbPRJjkBx', 'function': {'arguments': '{"query": "시그니처 스테이크"}', 'name': 'menu_search'}, 'type': 'function'}, {'id': 'call_kCetzxd55gyRND0vK0MeCIWy', 'function': {'arguments': '{"query": "스테이크"}', 'name': 'wine_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 136, 'total_tokens': 190, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BVcdYu6h1QjY6tAyKQ5sGD9P1aibz', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-0aa796a8-4059-4ee2-aba3-3819bf05ebe6-0', tool_calls=[{'name': 'menu_search', 'args': {'query': '시그니처 스테이크'}, 'id': 'call_QOxtPmNqByTnibexbPRJjkBx', 'type': 'tool_

## 여러 개의 도구를 사용하여 호출하기

In [45]:
tools = [tavily_search, wiki_summary_search, wine_search, menu_search]
for tool in tools:
    print(tool.name)

tavily_search_results_json
wiki_summary
wine_search
menu_search


In [46]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화
llm = ChatOpenAI(model="gpt-4o-mini")

# 4개의 검색 도구를 LLM에 바인딩
llm_with_tools = llm.bind_tools(tools=tools)

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def restaurant_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)

        if tool_call["name"] == "tavily_search_results_json":
            tool_message = tavily_search.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary_search.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wine_search":
            tool_message = wine_search.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "menu_search":
            tool_message = menu_search.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
response = restaurant_menu_chain.invoke("시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요.")

# 응답 출력
print(response.content)

menu_search: 
{'name': 'menu_search', 'args': {'query': '시그니처 스테이크'}, 'id': 'call_ayMoPXW8oijFw6w1DmR8hNIA', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
wine_search: 
{'name': 'wine_search', 'args': {'query': '스테이크와 어울리는 와인'}, 'id': 'call_ikwqTLE1xayGvTRHgCq6vsIS', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content="[Document(id='6f69f9f2-5598-4466-ada2-aa2520a1f3bc', metadata={'menu_name': '시그니처 스테이크', 'menu_number': 0, 'source': './data/restaurant_menu.txt'}, page_content='1. 시그니처 스테이크\\n   • 가격: ₩35,000\\n   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스\\n   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.'), Document(id='6c92e023-c6b6-4ce7-85d5-7de5a1bb2caf', metadata={'menu_name': '시그니처 스테이크', 'menu_number': 0,

In [47]:
# 체인 실행
response = restaurant_menu_chain.invoke("파스타 메뉴가 있나요? 이 음식의 역사 또는 유래를 알려주세요.")

# 응답 출력
print(response.content)

menu_search: 
{'name': 'menu_search', 'args': {'query': '파스타'}, 'id': 'call_I8sm7fqi5jAlfz0kOMMBF08K', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
wiki_summary: 
{'name': 'wiki_summary', 'args': {'query': '파스타'}, 'id': 'call_lC58gNdXqWuYkgjN57SXdA0F', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content="[Document(id='ceb2048a-fdd8-4553-b618-ab596d05209e', metadata={'menu_name': '해산물 파스타', 'menu_number': 5, 'source': './data/restaurant_menu.txt'}, page_content='6. 해산물 파스타\\n   • 가격: ₩24,000\\n   • 주요 식재료: 링귀네 파스타, 새우, 홍합, 오징어, 토마토 소스\\n   • 설명: 알 덴테로 삶은 링귀네 파스타에 신선한 해산물을 듬뿍 올린 메뉴입니다. 토마토 소스의 산미와 해산물의 감칠맛이 조화를 이루며, 마늘과 올리브 오일로 풍미를 더했습니다. 파슬리를 뿌려 향긋한 맛을 더합니다.'), Document(id='ce284568-4880-4454-9636-76212e6da84a', metadata={'menu_name': '해산물 파스타', 'menu_number': 5, 'source': './data/restaurant_menu.txt'}, page_

## Few-shot 프롬프트를 활용하여 ToolCalling 성능 개선하기

In [48]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate

examples = [
    HumanMessage("트러플 리조또의 가격과 특징, 그리고 어울리는 와인에 대해 알려주세요.", name="example_user"),
    AIMessage("메뉴 정보를 검색하고, 위키피디아에서 추가 정보를 찾은 후, 어울리는 와인을 검색해보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "search_menu", "args": {"query": "트러플 리조또"}, "id": "1"}]),
    ToolMessage("트러플 리조또: 가격 ₩28,000, 이탈리아 카나롤리 쌀 사용, 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리", tool_call_id="1"),
    AIMessage("트러플 리조또의 가격은 ₩28,000이며, 이탈리아 카나롤리 쌀을 사용하고 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리합니다. 이제 추가 정보를 위키피디아에서 찾아보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "wiki_summary", "args": {"query": "트러플 리조또", "k": 1}, "id": "2"}]),
    ToolMessage("트러플 리조또는 이탈리아 요리의 대표적인 리조또 요리 중 하나로, 고급 식재료인 트러플을 사용하여 만든 크리미한 쌀 요리입니다. 주로 아르보리오나 카나롤리 등의 쌀을 사용하며, 트러플 오일이나 생 트러플을 넣어 조리합니다. 리조또 특유의 크리미한 질감과 트러플의 강렬하고 독특한 향이 조화를 이루는 것이 특징입니다.", tool_call_id="2"),
    AIMessage("트러플 리조또의 특징에 대해 알아보았습니다. 이제 어울리는 와인을 검색해보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "search_wine", "args": {"query": "트러플 리조또에 어울리는 와인"}, "id": "3"}]),
    ToolMessage("트러플 리조또와 잘 어울리는 와인으로는 주로 중간 바디의 화이트 와인이 추천됩니다. 1. 샤르도네: 버터와 오크향이 트러플의 풍미를 보완합니다. 2. 피노 그리지오: 산뜻한 산미가 리조또의 크리미함과 균형을 이룹니다. 3. 베르나차: 이탈리아 토스카나 지방의 화이트 와인으로, 미네랄리티가 트러플과 잘 어울립니다.", tool_call_id="3"),
    AIMessage("트러플 리조또(₩28,000)는 이탈리아의 대표적인 리조또 요리 중 하나로, 이탈리아 카나롤리 쌀을 사용하고 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리합니다. 주요 특징으로는 크리미한 질감과 트러플의 강렬하고 독특한 향이 조화를 이루는 점입니다. 고급 식재료인 트러플을 사용해 풍부한 맛과 향을 내며, 주로 아르보리오나 카나롤리 등의 쌀을 사용합니다. 트러플 리조또와 잘 어울리는 와인으로는 중간 바디의 화이트 와인이 추천됩니다. 특히 버터와 오크향이 트러플의 풍미를 보완하는 샤르도네, 산뜻한 산미로 리조또의 크리미함과 균형을 이루는 피노 그리지오, 그리고 미네랄리티가 트러플과 잘 어울리는 이탈리아 토스카나 지방의 베르나차 등이 좋은 선택이 될 수 있습니다.", name="example_assistant"),
]

system = """You are an AI assistant providing restaurant menu information and general food-related knowledge.
For information about the restaurant's menu, use the search_menu tool.
For other general information, use the wiki_summary tool.
For wine recommendations or pairing information, use the search_wine tool.
If additional web searches are needed or for the most up-to-date information, use the search_web tool.
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    *examples,
    ("human", "{query}"),
])

# ChatOpenAI 모델 초기화
llm = ChatOpenAI(model="gpt-4o-mini")

# 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=tools)

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

# 체인 실행
query = "스테이크 메뉴가 있나요? 스테이크와 어울리는 와인을 추천해주세요."
response = fewshot_search_chain.invoke(query)

# 결과 출력
for tool_call in response.tool_calls:
    print(tool_call)

{'name': 'search_menu', 'args': {'query': '스테이크'}, 'id': 'call_yYyYlU6Xfs7LyaOO6yDrY8WG', 'type': 'tool_call'}
{'name': 'search_wine', 'args': {'query': '스테이크와 어울리는 와인'}, 'id': 'call_F52Z64duHVgHMt3ZIIAvfUNl', 'type': 'tool_call'}


In [49]:
# 체인 실행
query = "파스타의 유래에 대해서 알고 있나요? 서울 강남의 파스타 맛집을 추천해주세요."
response = fewshot_search_chain.invoke(query)

# 결과 출력
for tool_call in response.tool_calls:
    print(tool_call)

{'name': 'wiki_summary', 'args': {'query': '파스타의 유래'}, 'id': 'call_v4hFynv85ImTh7Vb475zpumK', 'type': 'tool_call'}
{'name': 'tavily_search_results_json', 'args': {'query': '서울 강남 파스타 맛집'}, 'id': 'call_6NEfee3e3nUEpIZPdw9qocXs', 'type': 'tool_call'}


In [50]:
from datetime import datetime
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain_openai import ChatOpenAI

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿
system = """You are an AI assistant providing restaurant menu information and general food-related knowledge.
For information about the restaurant's menu, use the search_menu tool.
For other general information, use the wiki_summary tool.
For wine recommendations or pairing information, use the search_wine tool.
If additional web searches are needed or for the most up-to-date information, use the search_web tool.
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system + f"Today's date is {today}."),
    *examples,
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화
llm = ChatOpenAI(model="gpt-4o-mini")

# 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=tools)

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def restaurant_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)

        if tool_call["name"] == "tavily_search_results_json":
            tool_message = tavily_search.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary_search.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wine_search":
            tool_message = wine_search.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "menu_search":
            tool_message = menu_search.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return fewshot_search_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


# 체인 실행
query = "스테이크 메뉴가 있나요? 스테이크와 어울리는 와인을 추천해주세요."
response = restaurant_menu_chain.invoke(query)

# 응답 출력
pprint(response.content)

menu_search: 
{'name': 'menu_search', 'args': {'query': 'steak'}, 'id': 'call_1RVaMEI4zgR53fx56rdhdaZF', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
wine_search: 
{'name': 'wine_search', 'args': {'query': 'red wine for steak'}, 'id': 'call_hoe0Z0AJDbKczhLzL2XHCk3r', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content="[Document(id='7455ce2b-7084-4b51-903b-4c4c7c87cc65', metadata={'menu_name': '안심 스테이크 샐러드', 'menu_number': 7, 'source': './data/restaurant_menu.txt'}, page_content='8. 안심 스테이크 샐러드\\n   • 가격: ₩26,000\\n   • 주요 식재료: 소고기 안심, 루꼴라, 체리 토마토, 발사믹 글레이즈\\n   • 설명: 부드러운 안심 스테이크를 얇게 슬라이스하여 신선한 루꼴라 위에 올린 메인 요리 샐러드입니다. 체리 토마토와 파마산 치즈 플레이크로 풍미를 더하고, 발사믹 글레이즈로 마무리하여 고기의 풍미를 한층 끌어올렸습니다.'), Document(id='85fbf828-bf7f-4240-bc56-eef1717b4df0', metadata={'menu_name': '안심 스테이크 샐러드', 'menu_number': 7, 'source': './da